# nRF
---

## 데이터 로드
---

In [1]:
# module 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.contrib import layers

import copy

In [2]:
plt.rcParams["figure.figsize"] = (14,8)

In [3]:
# dataset 로딩
x_train_load = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_activity_final_v3.csv').sort_values(by='acc_id', ascending=True)
y_train_load = pd.read_csv('C://Users//zeus_//Desktop//champion_data//train/train_label.csv').sort_values(by='acc_id', ascending=True)
x_test_load = pd.read_csv('C://Users//zeus_//Desktop//champion_data//test/test_activity_final_v3.csv').sort_values(by='acc_id', ascending=True)

In [4]:
x_data = x_train_load.drop(["Unnamed: 0"], axis=1)
y_data = y_train_load.label
x_test = x_test_load.drop(["Unnamed: 0"], axis=1)

In [5]:
mon2mon_train = pd.read_csv('C://Users//zeus_//Desktop//champion_data/mon2mon/feature148_train.csv')
mon2mon_test = pd.read_csv('C://Users//zeus_//Desktop//champion_data/mon2mon/feature148_test.csv')

In [6]:
x_data = pd.concat([x_data, mon2mon_train.drop(['Unnamed: 0', 'acc_id'], axis = 1)], axis= 1)
x_test = pd.concat([x_test, mon2mon_test.drop(['Unnamed: 0', 'acc_id'], axis = 1)], axis= 1)

In [7]:
x_data.to_csv('C://Users//zeus_//Desktop//champion_data//train/train_activity_final_v4.csv')
x_test.to_csv('C://Users//zeus_//Desktop//champion_data//test/test_activity_final_v4.csv')


### acc_id 제거

In [7]:
x_data = x_data.drop(['acc_id'], axis=1)

## 데이터셋 구분
---

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.3, random_state=0)

In [9]:
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size = 0.3, random_state = 0)

## 우리 모델
---

### 모델 1 - 랜덤포레스트

In [10]:
forest = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1, max_depth=25)

In [11]:
forest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [12]:
x_tree = []

for v in forest.estimators_:
    x_tree.append(v.predict(x_val))

In [13]:
x_tree = np.array(x_tree)

In [14]:
x_tree = x_tree.T

In [15]:
#한국어로 된 레이블을 숫자로 인코딩
idx2label = ['2month', 'month', 'retained', 'week']
label2idx = {c:i for i,c in enumerate(idx2label)}

#new y_val
y_val = [label2idx[v] for v in y_val]

### 모델 2 - 신경망

In [45]:
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.5, l1_ratio=1)
elastic_net.fit(x_tree, y_val)

ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=1,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [46]:
elastic_net.coef_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.00147786, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01285193, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.0007826 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [49]:
np.mean(f1_score(y_val, np.around(elastic_net.predict(x_tree)), average=None))

c:\python\venv\data_science\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.2645779726753956

In [51]:
X = tf.placeholder(tf.float32, [None, 100])
Y = tf.placeholder(tf.int32, [None])
keep_prob = tf.placeholder(tf.float32)

In [52]:
#Y_one_hot
Y_one_hot = tf.one_hot(Y, 4)

In [53]:
#Layer1
with tf.name_scope('layer1') as scope:
    w1 = tf.get_variable("weight1", [100, 64], tf.float32, layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([64]), name = "bias1")
    
    layer1 = tf.nn.relu(tf.matmul(X,w1) + b1)
    layer1 = tf.nn.dropout(layer1, keep_prob)

In [54]:
#Layer2
with tf.name_scope('layer2') as scope:
    w2 = tf.get_variable("weight2", [64, 32], tf.float32, layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([32]), name = "bias2")
    
    layer2 = tf.nn.relu(tf.matmul(layer1,w2) + b2)
    layer2 = tf.nn.dropout(layer2, keep_prob)

In [55]:
#Layer3
with tf.name_scope('layer3') as scope:
    w3 = tf.get_variable("weight3", [32, 16], tf.float32, layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([16]), name = "bias3")
    
    layer3 = tf.nn.relu(tf.matmul(layer2,w3) + b3)
    layer3 = tf.nn.dropout(layer3, keep_prob)

In [56]:
#Layer4
with tf.name_scope('layer4') as scope:
    w4 = tf.get_variable("weight4", [16, 8], tf.float32, layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([8]), name = "bias4")
    
    layer4 = tf.nn.relu(tf.matmul(layer3,w4) + b4)
    layer4 = tf.nn.dropout(layer4, keep_prob)

In [57]:
#hypothesis
with tf.name_scope('hypothesis') as scope:
    w5 = tf.get_variable("weight5", [8, 4], tf.float32, layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([4]), name = "bias5")
    
    logits = tf.matmul(layer4, w5) + b5

In [58]:
#Cost Function - softmax cross entropy
with tf.name_scope("cost") as scope:
    cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost)

In [59]:
#Train - Gradient Descent
with tf.name_scope("train") as scope:
    optimizer = tf.train.AdamOptimizer(0.01)
    train = optimizer.minimize(cost)

In [60]:
#Prediction
prediction = tf.argmax(logits, -1)

In [61]:
#Accuracy
with tf.name_scope("accuracy") as scope:
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, tf.argmax(Y_one_hot,-1)), tf.float32))

In [69]:
x_tree

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 0., 1., 1.],
       [3., 3., 3., ..., 3., 3., 3.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [2., 1., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 3., 2.]])

In [63]:
a = ""

In [67]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(100):
        
        avg_cost = 0
        total_batch = int(len(x_tree) / 100)
        
        for i in range(total_batch):
            batch_x = x_tree[i*100:(i+1)*100]
            batch_y = y_val[i*100:(i+1)*100]
            
            for i in range(5):
                cost_val, _ = sess.run([cost, train], feed_dict={X:batch_x,Y:batch_y, keep_prob:0.7})
                
            avg_cost += cost_val/total_batch
        
        print("[epoch : {}]  cost {}".format(epoch, avg_cost))
    
    print("accuracy : {}".format(sess.run(accuracy, feed_dict={X:x_tree, Y:y_val, keep_prob:1})))
    a = sess.run(prediction, feed_dict={X:x_tree, keep_prob:1})

[epoch : 0]  cost 1.0545503792308626
[epoch : 1]  cost 0.970876656827472
[epoch : 2]  cost 0.9507347930045352
[epoch : 3]  cost 0.9496096460592179
[epoch : 4]  cost 0.9466602336792721
[epoch : 5]  cost 0.9367885680425733
[epoch : 6]  cost 0.945371746733075
[epoch : 7]  cost 0.9436836160364601
[epoch : 8]  cost 0.93062547047933
[epoch : 9]  cost 0.9363433755579452
[epoch : 10]  cost 0.9240682295390534
[epoch : 11]  cost 0.9294501594134746
[epoch : 12]  cost 0.9300496524288541
[epoch : 13]  cost 0.9395095890476592
[epoch : 14]  cost 0.9417805929978685
[epoch : 15]  cost 0.9309320759205595
[epoch : 16]  cost 0.9681835841564901
[epoch : 17]  cost 0.9472227896962848
[epoch : 18]  cost 0.9368717556908018
[epoch : 19]  cost 0.9279453192438395
[epoch : 20]  cost 0.932652199552173
[epoch : 21]  cost 0.9339630291575476
[epoch : 22]  cost 0.9359056376275566
[epoch : 23]  cost 0.9541808837936046
[epoch : 24]  cost 0.9429947251365295
[epoch : 25]  cost 0.9388167829740612
[epoch : 26]  cost 0.922192

KeyboardInterrupt: 

In [89]:
a[16]

2

In [86]:
x_tree[16]

array([2., 3., 2., 3., 2., 3., 3., 1., 1., 2., 2., 3., 3., 0., 2., 3., 1.,
       2., 3., 2., 2., 1., 0., 2., 3., 2., 3., 0., 2., 1., 3., 3., 2., 3.,
       1., 0., 3., 0., 2., 3., 2., 0., 2., 3., 2., 0., 2., 0., 2., 2., 0.,
       3., 0., 1., 3., 3., 3., 2., 2., 0., 2., 0., 3., 3., 1., 1., 3., 3.,
       2., 3., 2., 0., 1., 3., 1., 1., 3., 1., 3., 2., 3., 2., 3., 1., 0.,
       2., 3., 2., 0., 2., 0., 0., 0., 3., 1., 3., 1., 2., 2., 2.])

In [87]:
y_val[16]

1

In [50]:
tf.reset_default_graph()

In [94]:
np.array([[1,2,3],[2,3,1],[3,2,1]]).shape

(3, 3)

In [106]:
c = np.array([[1,2,3], [2,1,3], [3,2,1]])

In [108]:
c.shape

(3, 3)

In [102]:
d = np.array([[100],[1000],[10000]])

In [110]:
np.dot(c,d).shape

(3, 1)

In [117]:
d = np.array([[1,2],[2,1],[1,1]])

In [118]:
d.shape

(3, 2)

In [130]:
for v in d:
#     v[0] = [1,0]
#     v[1] = [0,1]
    v[0] = "he"

ValueError: invalid literal for int() with base 10: 'he'

In [128]:
d

array([[2, 2],
       [2, 1],
       [2, 1]])

In [142]:
def sss(num):
    print(pd.crosstab(index = x_tree[num], columns='cnt'))
    print(y_val[num])

In [140]:
pd.crosstab(index = x_tree[104], columns='cnt')

col_0,cnt
row_0,
0.0,17
1.0,24
2.0,21
3.0,38


In [141]:
y_val[104]

2

In [17]:
sss(135)

NameError: name 'sss' is not defined